In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("train.csv")

In [3]:
new_df=df.sample(5000,random_state=2)

In [4]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,What is the best marketing automation tool for...,What is the best marketing automation tool for...,1
115086,115086,187729,187730,I am poor but I want to invest. What should I do?,I am quite poor and I want to be very rich. Wh...,0
327711,327711,454161,454162,I am from India and live abroad. I met a guy f...,T.I.E.T to Thapar University to Thapar Univers...,0
367788,367788,498109,491396,Why do so many people in the U.S. hate the sou...,My boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,Consequences of Bhopal gas tragedy?,What was the reason behind the Bhopal gas trag...,0


In [5]:
def preprocess(q):
    
    q = str(q).lower().strip()
    
    # Replace certain special characters with their string equivalents
    q = q.replace('%', ' percent')
    q = q.replace('$', ' dollar ')
    q = q.replace('₹', ' rupee ')
    q = q.replace('€', ' euro ')
    q = q.replace('@', ' at ')
    
    # The pattern '[math]' appears around 900 times in the whole dataset.
    q = q.replace('[math]', '')
    
    # Replacing some numbers with string equivalents (not perfect, can be done better to account for more cases)
    q = q.replace(',000,000,000 ', 'b ')
    q = q.replace(',000,000 ', 'm ')
    q = q.replace(',000 ', 'k ')
    q = re.sub(r'([0-9]+)000000000', r'\1b', q)
    q = re.sub(r'([0-9]+)000000', r'\1m', q)
    q = re.sub(r'([0-9]+)000', r'\1k', q)
    
    # Decontracting words
    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
    contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

    q_decontracted = []

    for word in q.split():
        if word in contractions:
            word = contractions[word]

        q_decontracted.append(word)

    q = ' '.join(q_decontracted)
    q = q.replace("'ve", " have")
    q = q.replace("n't", " not")
    q = q.replace("'re", " are")
    q = q.replace("'ll", " will")
    
    # Removing HTML tags
    q = BeautifulSoup(q)
    q = q.get_text()
    
    # Remove punctuations
    pattern = re.compile('\W')
    q = re.sub(pattern, ' ', q).strip()

    
    return q
    

In [6]:
new_df['question1'] = new_df['question1'].apply(preprocess)
new_df['question2'] = new_df['question2'].apply(preprocess)

In [7]:
new_df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [8]:
# Length of Question 1
new_df['q1_length'] = new_df['question1'].apply(len)

# Length of Question 2
new_df['q2_length'] = new_df['question2'].apply(len)

# Word Count of Question 1
new_df['q1_word_count'] = new_df['question1'].apply(lambda x: len(x.split()))

# Word Count of Question 2
new_df['q2_word_count'] = new_df['question2'].apply(lambda x: len(x.split()))

# Common Words Count
new_df['common_words_count'] = new_df.apply(lambda x: len(set(x['question1'].split()) & set(x['question2'].split())), axis=1)

# Unique Words Count
new_df['unique_words_count'] = new_df['question1'].apply(lambda x: len(set(x.split())))

# Character Count Difference
new_df['char_count_diff'] = new_df.apply(lambda x: abs(len(x['question1']) - len(x['question2'])), axis=1)

# Word Count Difference
new_df['word_count_diff'] = new_df.apply(lambda x: abs(len(x['question1'].split()) - len(x['question2'].split())), axis=1)

# Longest Common Subsequence (LCS) Length
def lcs_length(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    return dp[m][n]

new_df['lcs_length'] = new_df.apply(lambda x: lcs_length(x['question1'], x['question2']), axis=1)

# Jaccard Similarity
def jaccard_similarity(s1, s2):
    set1 = set(s1.split())
    set2 = set(s2.split())
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union

new_df['jaccard_similarity'] = new_df.apply(lambda x: jaccard_similarity(x['question1'], x['question2']), axis=1)


In [9]:
new_df

,id,qid1,qid2,question1,question2,is_duplicate,q1_length,q2_length,q1_word_count,q2_word_count,common_words_count,unique_words_count,char_count_diff,word_count_diff,lcs_length,jaccard_similarity
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12,13,1,0,75,0.857143
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,12,15,7,10,8,3,36,0.466667
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,26,20,3,20,15,6,47,0.090909
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,13,30,0,12,87,17,38,0.000000
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,3,5,15,4,25,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260459,260459,4753,83674,how did donald trump win the elections,how did donald trump got elected when there ar...,1,38,74,7,14,4,7,36,7,34,0.235294
126321,126321,93514,100300,what makes a person join isis,why do some people decide to join groups like ...,1,29,50,6,10,2,6,21,4,19,0.142857
280854,280854,13939,400517,why do people cheat on their partners,how do you forgive a cheating partner who is u...,0,37,57,7,10,1,7,20,3,23,0.062500
211893,211893,316824,316825,which religion invented the devil,what is the first religion to have a devil figure,0,33,49,5,10,3,5,16,5,24,0.250000


In [10]:
import fuzzywuzzy
from fuzzywuzzy import fuzz

# Advanced Token Features
new_df['token_set_ratio'] = new_df.apply(lambda x: fuzz.token_set_ratio(x['question1'], x['question2']), axis=1)
new_df['token_sort_ratio'] = new_df.apply(lambda x: fuzz.token_sort_ratio(x['question1'], x['question2']), axis=1)
new_df['partial_ratio'] = new_df.apply(lambda x: fuzz.partial_ratio(x['question1'], x['question2']), axis=1)
new_df['partial_token_set_ratio'] = new_df.apply(lambda x: fuzz.partial_token_set_ratio(x['question1'], x['question2']), axis=1)
new_df['partial_token_sort_ratio'] = new_df.apply(lambda x: fuzz.partial_token_sort_ratio(x['question1'], x['question2']), axis=1)
new_df['Q_ratio'] = new_df.apply(lambda x: fuzz.QRatio(x['question1'], x['question2']), axis=1)
new_df['UQ_ratio'] = new_df.apply(lambda x: fuzz.UQRatio(x['question1'], x['question2']), axis=1)
new_df['WRatio'] = new_df.apply(lambda x: fuzz.WRatio(x['question1'], x['question2']), axis=1)
new_df['UWRatio'] = new_df.apply(lambda x: fuzz.UWRatio(x['question1'], x['question2']), axis=1)

# Length-Based Features
new_df['length_diff'] = new_df.apply(lambda x: abs(len(x['question1']) - len(x['question2'])), axis=1)
new_df['word_length_diff'] = new_df.apply(lambda x: abs(len(x['question1'].split()) - len(x['question2'].split())), axis=1)
new_df['avg_length'] = new_df.apply(lambda x: (len(x['question1']) + len(x['question2'])) / 2, axis=1)
new_df['avg_word_length'] = new_df.apply(lambda x: (len(x['question1'].split()) + len(x['question2'].split())) / 2, axis=1)


# Displaying the modified DataFrame
new_df.head()


,id,qid1,qid2,question1,question2,is_duplicate,q1_length,q2_length,q1_word_count,q2_word_count,...,partial_token_set_ratio,partial_token_sort_ratio,Q_ratio,UQ_ratio,WRatio,UWRatio,length_diff,word_length_diff,avg_length,avg_word_length
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,100,99,99,99,99,99,1,0,75.5,13.0
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,12,15,...,100,64,69,69,70,70,8,3,52.0,13.5
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,26,20,...,100,35,26,26,41,41,15,6,111.5,23.0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,13,30,...,42,35,29,29,37,37,87,17,101.5,21.5
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,...,100,59,55,55,66,66,15,4,41.5,7.0


In [11]:
# Fuzzy Features

def fetch_fuzzy_features(row):
    
    q1 = row['question1']
    q2 = row['question2']
    
    fuzzy_features = [0.0]*4
    
    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

# Creating new feature columns for fuzzy features
new_df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
new_df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
new_df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
new_df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))

In [12]:
print(new_df.shape)
new_df.head()

(5000, 31)


,id,qid1,qid2,question1,question2,is_duplicate,q1_length,q2_length,q1_word_count,q2_word_count,...,Q_ratio,UQ_ratio,WRatio,UWRatio,length_diff,word_length_diff,avg_length,avg_word_length,fuzz_ratio,fuzz_partial_ratio
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,99,99,99,99,1,0,75.5,13.0,99,99
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,12,15,...,69,69,70,70,8,3,52.0,13.5,69,67
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,26,20,...,26,26,41,41,15,6,111.5,23.0,26,29
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,13,30,...,29,29,37,37,87,17,101.5,21.5,29,41
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,...,55,55,66,66,15,4,41.5,7.0,55,70


In [13]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
398782,what is the best marketing automation tool for...,what is the best marketing automation tool for...
115086,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...
327711,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...
367788,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...
151235,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy


In [14]:
final_df = new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

(5000, 26)


,is_duplicate,q1_length,q2_length,q1_word_count,q2_word_count,common_words_count,unique_words_count,char_count_diff,word_count_diff,lcs_length,...,Q_ratio,UQ_ratio,WRatio,UWRatio,length_diff,word_length_diff,avg_length,avg_word_length,fuzz_ratio,fuzz_partial_ratio
398782,1,75,76,13,13,12,13,1,0,75,...,99,99,99,99,1,0,75.5,13.0,99,99
115086,0,48,56,12,15,7,10,8,3,36,...,69,69,70,70,8,3,52.0,13.5,69,67
327711,0,104,119,26,20,3,20,15,6,47,...,26,26,41,41,15,6,111.5,23.0,26,29
367788,0,58,145,13,30,0,12,87,17,38,...,29,29,37,37,87,17,101.5,21.5,29,41
151235,0,34,49,5,9,3,5,15,4,25,...,55,55,66,66,15,4,41.5,7.0,55,70


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [16]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(5000, 6000)

In [17]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

(5000, 6026)


,is_duplicate,q1_length,q2_length,q1_word_count,q2_word_count,common_words_count,unique_words_count,char_count_diff,word_count_diff,lcs_length,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
398782,1,75,76,13,13,12,13,1,0,75,...,0,0,0,0,0,0,0,0,0,0
115086,0,48,56,12,15,7,10,8,3,36,...,0,0,0,0,0,0,0,0,0,0
327711,0,104,119,26,20,3,20,15,6,47,...,0,0,0,0,0,0,0,0,0,0
367788,0,58,145,13,30,0,12,87,17,38,...,1,0,0,0,0,0,0,0,0,0
151235,0,34,49,5,9,3,5,15,4,25,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0].values,test_size=0.2,random_state=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.71